In [7]:
# !pip install transformers

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoImageProcessor, Dinov2Model
from PIL import Image
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier
import argparse
import tarfile
import urllib.request
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from PIL import Image
from torch.utils.data import Dataset, Subset
from torchvision.transforms import v2

In [2]:
# class ImageDataset(Dataset):
#     """Simple dataset for loading images"""
    
#     def __init__(self, image_dir, image_list, labels=None, resolution=224):
#         """
#         Args:
#             image_dir: Directory containing images
#             image_list: List of image filenames
#             labels: List of labels (optional, for train/val)
#             resolution: Image resolution (96 for competition, 224 for DINO baseline)
#         """
#         self.image_dir = Path(image_dir)
#         self.image_list = image_list
#         self.labels = labels
#         self.resolution = resolution
    
#     def __len__(self):
#         return len(self.image_list)
    
#     def __getitem__(self, idx):
#         img_name = self.image_list[idx]
#         img_path = self.image_dir / img_name
        
#         # Load and resize image
#         image = Image.open(img_path).convert('RGB')
#         image = image.resize((self.resolution, self.resolution), Image.BILINEAR)
        
#         if self.labels is not None:
#             return image, self.labels[idx], img_name
#         return image, img_name

# class ImageDataset(Dataset):
#     """Simple dataset for loading images with MAE/DINO-style transforms."""

#     def __init__(self, image_dir, image_list, labels=None, resolution=224, split="train"):
#         """
#         Args:
#             image_dir: Directory containing images
#             image_list: List of filenames
#             labels: Optional list of labels
#             resolution: Base image size (224 for DINO, 96 for competition)
#             split: "train" or "val" or "test"
#         """
#         self.image_dir = Path(image_dir)
#         self.image_list = image_list
#         self.labels = labels
#         self.split = split
#         self.resolution = resolution

#         # Same ImageNet normalization used in CUBLinearProbeDataset
#         imagenet_mean = [0.485, 0.456, 0.406]
#         imagenet_std = [0.229, 0.224, 0.225]

#         if split == "train":
#             self.transform = v2.Compose([
#                 v2.RandomResizedCrop(resolution, scale=(0.8, 1.0)),
#                 v2.RandomHorizontalFlip(p=0.5),
#                 v2.ColorJitter(
#                     brightness=0.4,
#                     contrast=0.4,
#                     saturation=0.4,
#                     hue=0.1
#                 ),
#                 v2.ToImage(),
#                 v2.ToDtype(torch.float32, scale=True),
#                 v2.Normalize(mean=imagenet_mean, std=imagenet_std),
#             ])
#         else:
#             # val/test preprocessing
#             self.transform = v2.Compose([
#                 v2.Resize(256),
#                 v2.CenterCrop(resolution),
#                 v2.ToImage(),
#                 v2.ToDtype(torch.float32, scale=True),
#                 v2.Normalize(mean=imagenet_mean, std=imagenet_std),
#             ])

#     def __len__(self):
#         return len(self.image_list)

#     def __getitem__(self, idx):
#         img_name = self.image_list[idx]
#         img_path = self.image_dir / img_name

#         # Load RGB image
#         img = Image.open(img_path).convert('RGB')

#         # Apply SAME transforms that CUBLinearProbeDataset uses
#         img = self.transform(img)

#         if self.labels is not None:
#             return img, self.labels[idx], img_name
#         return img, img_name

class ImageDataset(Dataset):
    def __init__(self, image_dir, image_list, labels=None,
                 resolution=224, split="train", apply_transforms=True):
        self.image_dir = Path(image_dir)
        self.image_list = image_list
        self.labels = labels
        self.split = split
        self.resolution = resolution
        self.apply_transforms = apply_transforms

        imagenet_mean = [0.485, 0.456, 0.406]
        imagenet_std = [0.229, 0.224, 0.225]

        if apply_transforms:
            if split == "train":
                self.transform = v2.Compose([
                    v2.RandomResizedCrop(resolution, scale=(0.8, 1.0)),
                    v2.RandomHorizontalFlip(p=0.5),
                    v2.ColorJitter(
                        brightness=0.4,
                        contrast=0.4,
                        saturation=0.4,
                        hue=0.1
                    ),
                    v2.ToImage(),
                    v2.ToDtype(torch.float32, scale=True),
                    v2.Normalize(mean=imagenet_mean, std=imagenet_std),
                ])
            else:
                self.transform = v2.Compose([
                    v2.Resize(256),
                    v2.CenterCrop(resolution),
                    v2.ToImage(),
                    v2.ToDtype(torch.float32, scale=True),
                    v2.Normalize(mean=imagenet_mean, std=imagenet_std),
                ])
        else:
            self.transform = None   # <-- important

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_name = self.image_list[idx]
        img_path = self.image_dir / img_name

        img = Image.open(img_path).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)   # -> Tensor (for supervised)
        # else: keep img as PIL (for SSL)

        if self.labels is not None:
            return img, self.labels[idx], img_name
        return img, img_name



In [5]:
def collate_fn(batch):
    """Custom collate function to handle PIL images"""
    if len(batch[0]) == 3:  # train/val (image, label, filename)
        images = [item[0] for item in batch]
        labels = [item[1] for item in batch]
        filenames = [item[2] for item in batch]
        return images, labels, filenames
    else:  # test (image, filename)
        images = [item[0] for item in batch]
        filenames = [item[1] for item in batch]
        return images, filenames

In [6]:
! ls /home/long/code/amogh/data/testset_1

README.md	       test_images.csv		 train_labels.csv
sample_submission.csv  test_labels_INTERNAL.csv  val
test		       train			 val_labels.csv


In [7]:
from pathlib import Path
import pandas as pd
from torch.utils.data import DataLoader

# ============================================================
# Hyperparameters (replace args.*)
# ============================================================
batch_size = 64
num_workers = 4
resolution = 224   # or whatever you want for training
# ============================================================

# Load CSV files
data_dir = Path("/home/long/code/amogh/data/testset_1")

print("\nLoading dataset metadata...")
train_df = pd.read_csv(data_dir / 'train_labels.csv')
val_df = pd.read_csv(data_dir / 'val_labels.csv')
test_df = pd.read_csv(data_dir / 'test_labels_INTERNAL.csv')

print(f"  Train: {len(train_df)} images")
print(f"  Val:   {len(val_df)} images")
print(f"  Test:  {len(test_df)} images")
print(f"  Classes: {train_df['class_id'].nunique()}")

train_dataset1 = ImageDataset(
    data_dir / 'train',
    train_df['filename'].tolist(),
    train_df['class_id'].tolist(),
    resolution=resolution,
    apply_transforms=False,
)

val_dataset1 = ImageDataset(
    data_dir / 'val',
    val_df['filename'].tolist(),
    val_df['class_id'].tolist(),
    resolution=resolution,
    apply_transforms=False,
)

test_dataset1 = ImageDataset(
    data_dir / 'test',
    test_df['filename'].tolist(),
    labels=test_df['class_id'].tolist(),
    resolution=resolution,
    apply_transforms=False,
)

train_loader1 = DataLoader(
    train_dataset1,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    collate_fn=collate_fn,
)

val_loader1 = DataLoader(
    val_dataset1,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    collate_fn=collate_fn,
)

test_loader1 = DataLoader(
    test_dataset1,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    collate_fn=collate_fn,
)



Loading dataset metadata...
  Train: 8232 images
  Val:   1727 images
  Test:  1829 images
  Classes: 200


In [8]:
from pathlib import Path
import pandas as pd
from torch.utils.data import DataLoader

# ============================================================
# Hyperparameters (replace args.*)
# ============================================================
batch_size = 64
num_workers = 4
resolution = 224   # or whatever you want for training
# ============================================================

# Load CSV files
data_dir = Path("/home/long/code/amogh/data/testset_2")

print("\nLoading dataset metadata...")
train_df = pd.read_csv(data_dir / 'train_labels.csv')
val_df = pd.read_csv(data_dir / 'val_labels.csv')
test_df = pd.read_csv(data_dir / 'test_labels_INTERNAL.csv')

print(f"  Train: {len(train_df)} images")
print(f"  Val:   {len(val_df)} images")
print(f"  Test:  {len(test_df)} images")
print(f"  Classes: {train_df['class_id'].nunique()}")

train_dataset2 = ImageDataset(
    data_dir / 'train',
    train_df['filename'].tolist(),
    train_df['class_id'].tolist(),
    resolution=resolution,
    apply_transforms=False,
)

val_dataset2 = ImageDataset(
    data_dir / 'val',
    val_df['filename'].tolist(),
    val_df['class_id'].tolist(),
    resolution=resolution,
    apply_transforms=False,
)

test_dataset2 = ImageDataset(
    data_dir / 'test',
    test_df['filename'].tolist(),
    labels=test_df['class_id'].tolist(),
    resolution=resolution,
    apply_transforms=False,
)

train_loader2 = DataLoader(
    train_dataset2,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    collate_fn=collate_fn,
)

val_loader2 = DataLoader(
    val_dataset2,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    collate_fn=collate_fn,
)

test_loader2 = DataLoader(
    test_dataset2,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    collate_fn=collate_fn,
)



Loading dataset metadata...
  Train: 42000 images
  Val:   9000 images
  Test:  9000 images
  Classes: 100


In [9]:
from pathlib import Path
import pandas as pd
from torch.utils.data import DataLoader

# ============================================================
# Hyperparameters (replace args.*)
# ============================================================
batch_size = 64
num_workers = 4
resolution = 224   # or whatever you want for training
# ============================================================

# Load CSV files
data_dir = Path("/home/long/code/amogh/data/testset_3")

print("\nLoading dataset metadata...")
train_df = pd.read_csv(data_dir / 'train_labels.csv')
val_df = pd.read_csv(data_dir / 'val_labels.csv')
test_df = pd.read_csv(data_dir / 'test_labels_INTERNAL.csv')

print(f"  Train: {len(train_df)} images")
print(f"  Val:   {len(val_df)} images")
print(f"  Test:  {len(test_df)} images")
print(f"  Classes: {train_df['class_id'].nunique()}")

train_dataset3 = ImageDataset(
    data_dir / 'train',
    train_df['filename'].tolist(),
    train_df['class_id'].tolist(),
    resolution=resolution,
    apply_transforms=False,
)

val_dataset3 = ImageDataset(
    data_dir / 'val',
    val_df['filename'].tolist(),
    val_df['class_id'].tolist(),
    resolution=resolution,
    apply_transforms=False,
)

test_dataset3 = ImageDataset(
    data_dir / 'test',
    test_df['filename'].tolist(),
    labels=test_df['class_id'].tolist(),
    resolution=resolution,
    apply_transforms=False,
)

train_loader3 = DataLoader(
    train_dataset3,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    collate_fn=collate_fn
)

val_loader3 = DataLoader(
    val_dataset3,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    collate_fn=collate_fn
)

test_loader3 = DataLoader(
    test_dataset3,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    collate_fn=collate_fn
)



Loading dataset metadata...
  Train: 13895 images
  Val:   2977 images
  Test:  2978 images
  Classes: 397


In [10]:
# pip install lightly|

In [11]:
import tarfile
import urllib.request
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from PIL import Image
from torch.utils.data import Dataset, Subset
from torchvision.transforms import v2

In [12]:
import torch
import torchvision
from timm.models.vision_transformer import vit_base_patch32_224
from torch import nn
from lightly.models import utils
from lightly.models.modules import MAEDecoderTIMM, MaskedVisionTransformerTIMM
from lightly.transforms import MAETransform
import copy
from lightly.models.modules import DINOProjectionHead
from lightly.loss import DINOLoss  # only needed if you re-train SSL
from lightly.models.utils import deactivate_requires_grad

In [14]:
class DINO(nn.Module):
    def __init__(self, backbone, input_dim):
        super().__init__()
        self.student_backbone = backbone
        self.student_head = DINOProjectionHead(
            input_dim, 512, 64, 2048, freeze_last_layer=1
        )
        self.teacher_backbone = copy.deepcopy(backbone)
        self.teacher_head = DINOProjectionHead(input_dim, 512, 64, 2048)
        deactivate_requires_grad(self.teacher_backbone)
        deactivate_requires_grad(self.teacher_head)

    def forward(self, x):
        y = self.student_backbone(x).flatten(start_dim=1)
        z = self.student_head(y)
        return z

    def forward_teacher(self, x):
        y = self.teacher_backbone(x).flatten(start_dim=1)
        z = self.teacher_head(y)
        return z


In [15]:
import torchvision

# --- Build same backbone as used for DINO pretraining ---
resnet = torchvision.models.resnet18()
# resnet = torchvision.models.resnet34()
backbone = nn.Sequential(*list(resnet.children())[:-1])  # (B, 512, 1, 1)
input_dim = 512

dino_model = DINO(backbone, input_dim)

# --- Load your pre-trained DINO checkpoint ---
ckpt = torch.load(
    "/home/long/code/dl_project1/experiments/outputs/dino-v1/dino-v1_small_100.pt",
    map_location="cpu",
)

dino_model.load_state_dict(ckpt["model_state"], strict=True)



/home/long/code/environments/wejepa/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


<All keys matched successfully>

In [16]:
# 1. Extract raw subsets from train_ds and val_ds
# raw_train_subset = train_raw_ds
# raw_val_subset   = val_raw_ds

# 2. SSL transform
from lightly.transforms import MAETransform
# ssl_transform = MAETransform()
from lightly.transforms.dino_transform import DINOTransform

# ssl_transform = DINOTransform()
# ssl_transform = get_cub_transform(split="train", img_size=224)
ssl_transform = DINOTransform()

# 3. SSL dataset wrapper
class CUB_SSL_Dataset(torch.utils.data.Dataset):
    def __init__(self, subset, transform):
        self.subset = subset
        self.transform = transform

    def __len__(self):
        return len(self.subset)

    # def __getitem__(self, idx):
    #     data = self.subset[idx]  # (img, label, path)
    #     img = data[0]
    #     return self.transform(img)
    def __getitem__(self, idx):
        img, _, _ = self.subset[idx]
        # import pdb
        # pdb.set_trace()
        return self.transform(img)  # returns list[Tensor] from DINOTransform

In [17]:
# 4. Combine train + val subsets (no labels)
from torch.utils.data import ConcatDataset
ssl_trainval_raw = ConcatDataset([
    train_dataset1, val_dataset1,
    train_dataset2, val_dataset2,
    train_dataset3, val_dataset3,
])


# 5. Build SSL dataset
ssl_trainval_ds = CUB_SSL_Dataset(
    subset=ssl_trainval_raw,
    transform=ssl_transform,
)

# 6. Build SSL dataloader
ssl_loader = DataLoader(
    ssl_trainval_ds,
    batch_size=64,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True,
    # collate_fn=collate_fn
)

print("SSL training images:", len(ssl_trainval_ds))

if ssl_loader.dataset is not None:
    print(f"Dataset found. Total samples: {len(ssl_loader.dataset)}")
    print(f"Batch size: {ssl_loader.batch_size}")
    print(f"Number of workers: {ssl_loader.num_workers}")
else:
    print("Dataset is missing.")

SSL training images: 77831
Dataset found. Total samples: 77831
Batch size: 64
Number of workers: 4


In [143]:
# print("===== SSL Train+Val Raw Dataset =====")
# print("Type:", type(ssl_trainval_raw))
# print("Total length:", len(ssl_trainval_raw))

# print("\n-- Sub-datasets inside ConcatDataset --")
# for i, ds in enumerate(ssl_trainval_raw.datasets):
#     print(f"[{i}] {type(ds)}, length = {len(ds)}")


In [18]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [19]:
from lightly.loss import DINOLoss
from lightly.models.modules import DINOProjectionHead
from lightly.models.utils import deactivate_requires_grad, update_momentum
from lightly.transforms.dino_transform import DINOTransform
from lightly.utils.scheduler import cosine_schedule

In [20]:
# ---- Inspect one batch from SSL dataloader ----

batch = next(iter(ssl_loader))

print("\n=== SSL Batch Debug ===")

if isinstance(batch, list) or isinstance(batch, tuple):
    print(f"Batch is a {type(batch)} with length {len(batch)}")

# DINOTransform returns a list of views per item, but DataLoader collates it into:
# batch = list_of_views, where each element has shape (B, C, H, W)

# Example: batch[0] = global crops   shape: (B, 3, 224, 224)
#          batch[1] = global crops   shape: (B, 3, 224, 224)
#          batch[2] = local crops    shape: (B, 3, 96, 96)
# etc.

for i, view in enumerate(batch):
    print(f"\n--- View {i} ---")
    print(f"Type: {type(view)}")
    try:
        print(f"Shape: {view.shape}")
    except Exception:
        print("View has no `.shape` attribute")
    print(f"Dtype: {getattr(view, 'dtype', None)}")

print("\n=== End Debug ===\n")



=== SSL Batch Debug ===
Batch is a <class 'list'> with length 8

--- View 0 ---
Type: <class 'torch.Tensor'>
Shape: torch.Size([64, 3, 224, 224])
Dtype: torch.float32

--- View 1 ---
Type: <class 'torch.Tensor'>
Shape: torch.Size([64, 3, 224, 224])
Dtype: torch.float32

--- View 2 ---
Type: <class 'torch.Tensor'>
Shape: torch.Size([64, 3, 96, 96])
Dtype: torch.float32

--- View 3 ---
Type: <class 'torch.Tensor'>
Shape: torch.Size([64, 3, 96, 96])
Dtype: torch.float32

--- View 4 ---
Type: <class 'torch.Tensor'>
Shape: torch.Size([64, 3, 96, 96])
Dtype: torch.float32

--- View 5 ---
Type: <class 'torch.Tensor'>
Shape: torch.Size([64, 3, 96, 96])
Dtype: torch.float32

--- View 6 ---
Type: <class 'torch.Tensor'>
Shape: torch.Size([64, 3, 96, 96])
Dtype: torch.float32

--- View 7 ---
Type: <class 'torch.Tensor'>
Shape: torch.Size([64, 3, 96, 96])
Dtype: torch.float32

=== End Debug ===



In [147]:
device

'cuda:0'

In [148]:
batch = next(iter(ssl_loader))  # or ssl_loader

print("Type of batch:", type(batch))

if isinstance(batch, (list, tuple)):
    print("Batch length:", len(batch))
    for i, x in enumerate(batch):
        print(f"  item[{i}] type: {type(x)}")
        if hasattr(x, "shape"):
            print(f"  item[{i}] shape:", x.shape)
else:
    print("Batch shape:", batch.shape)


Type of batch: <class 'list'>
Batch length: 8
  item[0] type: <class 'torch.Tensor'>
  item[0] shape: torch.Size([64, 3, 224, 224])
  item[1] type: <class 'torch.Tensor'>
  item[1] shape: torch.Size([64, 3, 224, 224])
  item[2] type: <class 'torch.Tensor'>
  item[2] shape: torch.Size([64, 3, 96, 96])
  item[3] type: <class 'torch.Tensor'>
  item[3] shape: torch.Size([64, 3, 96, 96])
  item[4] type: <class 'torch.Tensor'>
  item[4] shape: torch.Size([64, 3, 96, 96])
  item[5] type: <class 'torch.Tensor'>
  item[5] shape: torch.Size([64, 3, 96, 96])
  item[6] type: <class 'torch.Tensor'>
  item[6] shape: torch.Size([64, 3, 96, 96])
  item[7] type: <class 'torch.Tensor'>
  item[7] shape: torch.Size([64, 3, 96, 96])


In [ ]:
# ---------- Project / save dir ----------
PROJECT_NAME = "dino-v1"  # folder name
save_dir = "/home/long/code/amogh/data/models/"
save_dir = Path(save_dir)
save_dir.mkdir(parents=True, exist_ok=True)
print(f"Save directory ready: {save_dir}")

device = "cuda" if torch.cuda.is_available() else "cpu"
dino_model.to(device)

ckpt_path = save_dir / f"{PROJECT_NAME}_full_finetuned_18.pt"
start_epoch = 0

# # --- Optimizer: only student parameters ---
# optimizer = torch.optim.AdamW(
#     list(dino_model.student_backbone.parameters()) +
#     list(dino_model.student_head.parameters()),
#     lr=1.5e-4,
#     weight_decay=1e-4,
# )

optimizer = torch.optim.Adam(dino_model.parameters(), lr=0.001)

criterion = DINOLoss(
    output_dim=2048,
    warmup_teacher_temp_epochs=5,
)
# move loss to correct device because it also contains parameters
criterion = criterion.to(device)

# ---------- Training loop ----------
import time
from tqdm import tqdm

num_epochs = 100
print("Starting SSL Training (DINO)")

global_step = 0

for epoch in range(start_epoch, num_epochs):
    dino_model.train()
    total_loss = 0.0
    n_samples = 0

    epoch_start = time.time()

    # EMA momentum for teacher this epoch
    momentum_val = cosine_schedule(epoch, num_epochs, 0.996, 1.0)

    for batch in tqdm(ssl_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):

        # ssl_loader should give a list of crops (views)
        # Handle both "views" and "(views, _)" cases
        if isinstance(batch, (list, tuple)) and isinstance(batch[0], torch.Tensor):
            views = batch
        elif isinstance(batch, (list, tuple)):
            views = batch[0]
        else:
            views = batch

        # move all crops to GPU
        views = [v.to(device, non_blocking=True) for v in views]

        # ---- EMA update for teacher ----
        update_momentum(dino_model.student_backbone, dino_model.teacher_backbone, m=momentum_val)
        update_momentum(dino_model.student_head,     dino_model.teacher_head,     m=momentum_val)

        # first two are global crops for teacher
        global_views = views[:2]

        # teacher on global crops (no grad)
        with torch.no_grad():
            teacher_out = [dino_model.forward_teacher(v) for v in global_views]

        # student on all crops (global + local)
        student_out = [dino_model(v) for v in views]

        # ---- DINO loss ----
        loss = criterion(teacher_out, student_out, epoch=epoch)

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        dino_model.student_head.cancel_last_layer_gradients(current_epoch=epoch)
        optimizer.step()

        bsz = views[0].size(0)
        total_loss += loss.item() * bsz
        n_samples += bsz
        global_step += 1

    avg_loss = total_loss / max(n_samples, 1)
    elapsed = time.time() - epoch_start

    print(f"Epoch {epoch:02d} | train loss: {avg_loss:.5f} | time: {elapsed:.1f}s")

    if (epoch + 1) % 1 == 0:
        # ---- Save checkpoint (always same filename) ----
        ckpt = {
            "epoch": epoch,
            "model_state": dino_model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
            "avg_loss": avg_loss,
        }
        torch.save(ckpt, ckpt_path)
        print(f"✓ Saved checkpoint: {ckpt_path}")

Save directory ready: /home/long/code/amogh/data/models
Starting SSL Training (DINO)


Epoch 1/100: 100%|█████████████████████████████████████████████████████████████████████████████████| 1217/1217 [06:04<00:00,  3.34it/s]


Epoch 00 | train loss: 2.45309 | time: 364.8s
✓ Saved checkpoint: /home/long/code/amogh/data/models/dino-v1_full_finetuned_18.pt


Epoch 2/100: 100%|█████████████████████████████████████████████████████████████████████████████████| 1217/1217 [06:05<00:00,  3.33it/s]


Epoch 01 | train loss: 2.08134 | time: 365.9s
✓ Saved checkpoint: /home/long/code/amogh/data/models/dino-v1_full_finetuned_18.pt


Epoch 3/100: 100%|█████████████████████████████████████████████████████████████████████████████████| 1217/1217 [06:07<00:00,  3.31it/s]


Epoch 02 | train loss: 2.05473 | time: 367.9s
✓ Saved checkpoint: /home/long/code/amogh/data/models/dino-v1_full_finetuned_18.pt


Epoch 4/100: 100%|█████████████████████████████████████████████████████████████████████████████████| 1217/1217 [06:06<00:00,  3.32it/s]


Epoch 03 | train loss: 2.02415 | time: 366.4s
✓ Saved checkpoint: /home/long/code/amogh/data/models/dino-v1_full_finetuned_18.pt


Epoch 5/100:  24%|███████████████████▍                                                              | 288/1217 [01:36<03:54,  3.95it/s]